# A1: Exercise – ver. 0420-2

최규빈  
2025-01-01

In [1]:
import torch
import torchvision
import PIL
import pandas as pd

# `#` – 최적화

## `$`. 경사하강법, 경사상승법

`(1)` 아래의 함수를 최소화하는 $x$를 경사하강법기반의 알고리즘을
활용하여 추정하라. (꼭 SGD를 쓸 필요는 없음)

$$f(x)=(x-1)^2$$

초기값은 $x=3$ 으로 설정하라.

`-` 함수 선언

In [2]:
## 본함수
def f(x) :
    return (x-1)**2

## 미분계수 산출
def f_prime(x) :
    h = 0.000001
    return (f(x+h) - f(x))/h

In [81]:
## 경사하강법
def GD(x, f_prime, lr = 0.1, convex = True) :
    slope = f_prime(x)
    
    if (abs(slope) <= 1e-6) :
        print(f"optimal value x = {x:.4f}")
        
        return x

    else :
        if convex :
            renewed_x = x - lr*slope
        else :
            renewed_x = x + lr*slope

        return SGD(renewed_x, f_prime, lr, convex)

In [86]:
optim = GD(x = 3, f_prime = f_prime)

optimal value x = 1.0000


`(2)` 아래의 함수를 최대화하는 $x$를 경사하강법기반의 알고리즘을
활용하여 추정하라. (꼭 SGD를 쓸 필요는 없음)

$$f(x)=-x^2 +6x-9 $$

초기값은 $x=0$ 으로 설정하라.

hint: $f(x)$을 최대화하는 $x$는 $-f(x)$를 최소화한다.

In [72]:
def g(x) :
    return (-x**2 + 6*x - 9)

def g_prime(x) :
    h = 0.000001
    return (g(x+h) - g(x))/h

In [84]:
optim = SGD(x = 0.0, f_prime = g_prime, lr = 0.1, convex = False)

optimal value x = 3.0000


## `$`. 정규분포 MLE

아래는 $X_i \sim N(3, 2^2)$ 를 생성하는 코드이다.

In [124]:
torch.manual_seed(43052)
x = torch.randn((10000,1)) * 2 + 3

함수 $l(\mu, \sigma)$를 최대화하는 $(\mu, \sigma)$를 경사하강법기반의
알고리즘을 활용하여 추정하라. (꼭 SGD를 쓸 필요는 없음)

$$l(\mu, \sigma) = \sum_{i=1}^{n} \log f(x_i), \quad
f(x_i) = \frac{1}{\sqrt{2\pi}\sigma} \exp\left(-\frac{(x_i - \mu)^2}{2\sigma^2}\right)$$

* 위는 아래 함수를 최소화하는 $\mu, \sigma$를 찾는 것과 동일하므로

$$l_2(\mu, \sigma) = -\frac1n\sum_{i=1}^{n} \log f(x_i)$$

In [178]:
def neg_l(What) :
    fx = (1/((2*torch.pi)**0.5 * What[1])) * torch.exp(-(x - What[0])**2 / (2*What[1]**2))

    return -torch.mean(torch.log(fx))

In [187]:
What = torch.tensor([0.0, 1.0], requires_grad = True)

##---##
for _ in range(1000) :
    loss = neg_l(What)
    loss.backward()
    What.data -= 0.1*What.grad
    What.grad = What.grad*0

In [193]:
print(f"추정된 mu = {What.data[0]:.4f}\n추정된 sigma = {What.data[1]:.4f}")

추정된 mu = 3.0187
추정된 sigma = 2.0113


## `$`. 베르누이 MLE

아래는 $X_i \overset{iid}{\sim} Ber(0.8)$을 생성하는 코드이다.

In [194]:
torch.manual_seed(43052)
x = torch.bernoulli(torch.tensor([0.8]*10000)).reshape(-1,1)

함수 $l(p)$를 최대화하는 $p$를 경사하강법기반을 알고리즘을 활용하여
추정하라. (꼭 SGD를 쓸 필요는 없음)

$$l(p) = \sum_{i=1}^{n} \log f(x_i), \quad f(x_i) = p^{x_i} (1-p)^{1-x_i}$$

* 위는 아래 함수를 최소화하는 $\mu, \sigma$를 찾는 것과 동일하므로

$$l_2(p) = -\frac1n\sum_{i=1}^{n} \log f(x_i)$$

In [196]:
def neg_l(p) :
    fx = p**x * (1-p)**(1-x)

    return -(torch.mean(torch.log(fx)))

In [224]:
phat = torch.tensor([0.5], requires_grad = True)

##---##
for _ in range(1000) :
    loss = neg_l(phat)
    loss.backward()
    phat.data -= 0.01*phat.grad
    phat.grad = phat.data*0

In [229]:
print(f"추정된 p = {phat.data[0]:.4f}")

추정된 p = 0.7971


## `$`. 회귀모형의 MLE

아래의 모형을 생각하자.

-   $Y_i \overset{iid}{\sim} \mathcal{N}(\mu_i, 1)$
-   $\mu_i = \beta_0 + \beta_1 x_i = 0.5 + 2x_i$

아래는 위의 모형에서 얻은 샘플이다.

In [230]:
x = torch.linspace(0,1,10000).reshape(10000,1)
y = 0.5+2*x + torch.randn(10000,1)

함수 $l(\beta_0, \beta_1)$를 최대화하는 $(\beta_0, \beta_1)$를
경사하강법기반의 알고리즘을 활용하여 추정하라. (꼭 SGD를 쓸 필요는 없음)

$$
l(\beta_0, \beta_1) = \sum_{i=1}^{n} \log f(y_i), \quad f(y_i) = \frac{1}{\sqrt{2\pi}} e^{-\frac{1}{2}(y_i - \mu_i)^2}, \quad \mu_i = \beta_0 + \beta_1 x_i
$$

* 위는 아래 함수를 최소화하는 $\mu, \sigma$를 찾는 것과 동일하므로

$$l_2(\beta_0, \beta_1) = -\frac1n\sum_{i=1}^{n} \log f(x_i)$$

In [233]:
def neg_l(Bhat) :
    mu = Bhat[0] + Bhat[1] * x
    fy = (1/((2*torch.pi)**0.5)) * torch.exp(-0.5*(y - mu)**2)

    return -torch.mean(torch.log(fy))

In [237]:
Bhat = torch.tensor([0.0, 0.0], requires_grad = True)

##---##
for _ in range(1000) :
    loss = neg_l(Bhat)
    loss.backward()
    Bhat.data -= 0.1*Bhat.grad
    Bhat.grad = Bhat.data*0

In [239]:
print(f"추정된 beta0 = {Bhat.data[0]:.4f}\n추정된 beta1 = {Bhat.data[1]:.4f}")

추정된 beta0 = 0.5039
추정된 beta1 = 2.0019


## `$`. 로지스틱모형의 MLE

아래의 모형을 생각하자.

-   $Y_i \overset{iid}{\sim} Ber(\pi_i)$
-   $\pi_i = \dfrac{\exp(w_0 + w_1 x_i)}{1 + \exp(w_0 + w_1 x_i)} = \dfrac{\exp(-1 + 0.5x_i)}{1 + \exp(-1 + 0.5x_i)}$

아래는 위의 모형에서 얻은 샘플이다.

In [242]:
x = torch.linspace(-1,1,10000).reshape(10000,1)
pi = torch.exp(-1 + 0.5* x) / (1 + torch.exp(-1 + 0.5 * x))
y = torch.bernoulli(pi)

함수 $l(w_0, w_1)$을 최대화하는 파라미터 $(w_0, w_1)$를 경사하강법기반을
알고리즘을 활용하여 추정하라. (꼭 SGD를 쓸 필요는 없음)

$$
l(w_0, w_1) = \sum_{i=1}^{n} \log f(y_i), \quad
f(x_i) = \pi_i^{y_i}(1 - \pi_i)^{1 - y_i}, \quad
\pi_i = \dfrac{\exp(w_0 + w_1 x_i)}{1 + \exp(w_0 + w_1 x_i)}
$$

* 위는 아래 함수를 최소화하는 $\mu, \sigma$를 찾는 것과 동일하므로

$$l_2(w_0, w_1) = -\frac1n\sum_{i=1}^{n} \log f(x_i)$$

In [250]:
def neg_l(What) :
    pii = torch.exp(What[0] + What[1] * x)/(1+torch.exp(What[0] + What[1] * x))
    fx = pii**y * (1-pii)**(1-y)

    return -torch.mean(torch.log(fx))

In [253]:
What = torch.tensor([0.0, 0.0], requires_grad = True)

##---##
for _ in range(1000) :
    loss = neg_l(What)
    loss.backward()
    What.data -= 0.1*What.grad
    What.grad = What.data*0

In [255]:
print(f"추정된 w0 = {What.data[0]:.4f}\n추정된 w1 = {What.data[1]:.4f}")

추정된 w0 = -1.0231
추정된 w1 = 0.5272


# `#` – 분류

## `$`. iris

아래의 자료를 고려하자.

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/guebin/DL2025/main/posts/iris.csv")
df

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


위의 자료는 아이리스 데이터셋으로 머신러닝에서 자주 사용되는
분류(classification) 예제 데이터이다. 데이터는 다음과 같은 특징을 가지고
있다:

**샘플 수: 150개**

**특징 수: 4개**

-   꽃받침 길이 (sepal length)
-   꽃받침 너비 (sepal width)
-   꽃잎 길이 (petal length)
-   꽃잎 너비 (petal width)

**클래스 수: 3개 (각 50개 샘플)**

-   0: setosa
-   1: versicolor
-   2: virginica

`(1)` 주어진 데이터를 8:2 비율로 학습용(df_train)과
테스트용(df_test)으로 나누고, SepalLength, SepalWidth, PetalLength,
PetalWidth를 입력으로 하여 Species를 예측할 수 있도록 데이터를 텐서
형태로 변환하라.

hint: 아래의 코드를 활용할 것

``` python
df_train = df.sample(frac=0.8, random_state=42)
df_test = df.drop(df_train.index)
#---#
X = torch.tensor(df_train.iloc[:,:4].values).float()
y = ???
XX = ???
yy = ???
```

In [3]:
df_train = df.sample(frac=0.8, random_state=42)
df_test = df.drop(df_train.index)
#---#
X = torch.tensor(df_train.iloc[:,:4].values).float()
y = torch.nn.functional.one_hot(torch.tensor(df_train.Species.values).int().long()).float()
XX = torch.tensor(df_test.iloc[:,:4].values).float()
yy = torch.nn.functional.one_hot(torch.tensor(df_test.Species.values).int().long()).float()

`(2)` 아래의 제약사항에 맞추어 Species를 예측할 수 있는 적당한
네트워크를 학습하라.

**제약사항**

-   학습가능한 파라메터는 1층만 설계할 것
-   학습 후 test accuracy 가 70% 이상일것

In [4]:
net = torch.nn.Sequential(
    torch.nn.Linear(4, 3) ## 레이어 한개
)

loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters(), lr = 0.01)

##--##
for epoc in range(500) :
    net.train()
    netout = net(X)
    loss = loss_fn(netout, y)
    loss.backward()
    optimizr.step()
    optimizr.zero_grad()

    if epoc % 50 == 0 :
        net.eval()
        acc = (net(X).argmax(axis = 1) == y.argmax(axis = 1)).float().mean().item()
        print(f"epoc : {epoc},\ttrain acc = {acc:.4f}")

epoc : 0,	train acc = 0.3167
epoc : 50,	train acc = 0.7417
epoc : 100,	train acc = 0.8250
epoc : 150,	train acc = 0.9167
epoc : 200,	train acc = 0.9250
epoc : 250,	train acc = 0.9417
epoc : 300,	train acc = 0.9500
epoc : 350,	train acc = 0.9583
epoc : 400,	train acc = 0.9583
epoc : 450,	train acc = 0.9583


In [5]:
acc = (net(XX).argmax(axis = 1) == yy.argmax(axis = 1)).float().mean().item()
print(f"test acc = {acc:.4f}")

test acc = 0.9667


> ?????? 이게 말이 됨??? -> 애초에 선형 분류문제였던 것 같음...

`(3)` 아래의 제약사항에 맞추어 Species를 예측할 수 있는 적당한
네트워크를 학습하라.

**제약사항**

-   학습가능한 파라메터는 1층만 설계할 것
-   학습 후 test accuracy 가 70% 이상일것
-   train/test에서 모두 `batch_size = 10` 으로 설정할 것
-   GPU를 사용할 것

In [8]:
## data
ds_train = torch.utils.data.TensorDataset(X, y)
dl_train = torch.utils.data.DataLoader(ds_train, batch_size = 10, shuffle = True)

ds_test = torch.utils.data.TensorDataset(XX, yy)
dl_test = torch.utils.data.DataLoader(ds_test, batch_size = 10, shuffle = True)

## network
net = torch.nn.Sequential(
    torch.nn.Linear(4, 3) ## 레이어 한개
).to("cuda:0")

loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters(), lr = 0.01)

##--##
for epoc in range(50) :
    net.train()

    for Xm, ym in dl_train :
        Xm = Xm.to("cuda:0")
        ym = ym.to("cuda:0")
        
        netout = net(Xm)
        loss = loss_fn(netout, ym)
        loss.backward()
        optimizr.step()
        optimizr.zero_grad()

    if epoc % 5 == 0 :
        net.eval()

        s = 0
        
        for Xm, ym in dl_train :
            Xm = Xm.to("cuda:0")
            ym = ym.to("cuda:0")
            s += (net(Xm).argmax(axis = 1) == ym.argmax(axis = 1)).sum().item()

        acc = s/len(y)
        print(f"epoch : {epoc},\ttrain_acc = {acc:.4f}")

epoch : 0,	train_acc = 0.3583
epoch : 5,	train_acc = 0.9333
epoch : 10,	train_acc = 0.9417
epoch : 15,	train_acc = 0.9167
epoch : 20,	train_acc = 0.9250
epoch : 25,	train_acc = 0.9083
epoch : 30,	train_acc = 0.9583
epoch : 35,	train_acc = 0.9500
epoch : 40,	train_acc = 0.9667
epoch : 45,	train_acc = 0.9667


In [9]:
s = 0

for XXm, yym in dl_test :
    XXm = XXm.to("cuda:0")
    yym = yym.to("cuda:0")

    s += (net(XXm).argmax(axis = 1) == yym.argmax(axis = 1)).sum()

acc = s/len(yy)
print(f"test_acc = {acc:.4f}")

test_acc = 0.9667


`(4)` 아래의 제약사항에 맞추어 Species를 예측할 수 있는 적당한
네트워크를 학습하라.

**제약사항**

-   학습가능한 파라메터는 1층만 설계할 것
-   학습 후 test accuracy 가 70% 이상일 것
-   train에서 `batch_size = 50` 으로, test에서는 `batch_size=30`을
    설정할것
-   GPU를 사용할 것
-   매 epoch마다 loss와 train accuracy를 출력할 것

In [15]:
## data
ds_train = torch.utils.data.TensorDataset(X, y)
dl_train = torch.utils.data.DataLoader(ds_train, batch_size = 50, shuffle = True)

ds_test = torch.utils.data.TensorDataset(XX, yy)
dl_test = torch.utils.data.DataLoader(ds_test, batch_size = 30, shuffle = True)

## network
net = torch.nn.Sequential(
    torch.nn.Linear(4, 3) ## 레이어 한개
).to("cuda:0")

loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters(), lr = 0.01)

##--##
for epoc in range(50) :
    net.train()

    for Xm, ym in dl_train :
        Xm = Xm.to("cuda:0")
        ym = ym.to("cuda:0")
        
        netout = net(Xm)
        loss = loss_fn(netout, ym)
        loss.backward()
        optimizr.step()
        optimizr.zero_grad()

    net.eval()

    s = 0
    
    for Xm, ym in dl_train :
        Xm = Xm.to("cuda:0")
        ym = ym.to("cuda:0")
        s += (net(Xm).argmax(axis = 1) == ym.argmax(axis = 1)).sum().item()

    acc = s/len(y)
    print(f"epoch : {epoc},\tloss = {loss:.4f},\ttrain_acc = {acc:.4f}")

epoch : 0,	loss = 1.8602,	train_acc = 0.3250
epoch : 1,	loss = 2.1620,	train_acc = 0.3083
epoch : 2,	loss = 1.6821,	train_acc = 0.2167
epoch : 3,	loss = 2.1459,	train_acc = 0.2833
epoch : 4,	loss = 1.1585,	train_acc = 0.2833
epoch : 5,	loss = 1.2899,	train_acc = 0.2833
epoch : 6,	loss = 1.2899,	train_acc = 0.2917
epoch : 7,	loss = 1.1892,	train_acc = 0.2833
epoch : 8,	loss = 1.1405,	train_acc = 0.2500
epoch : 9,	loss = 1.0963,	train_acc = 0.3000
epoch : 10,	loss = 1.0746,	train_acc = 0.5333
epoch : 11,	loss = 1.0237,	train_acc = 0.6167
epoch : 12,	loss = 0.9573,	train_acc = 0.7333
epoch : 13,	loss = 0.9723,	train_acc = 0.7583
epoch : 14,	loss = 0.8908,	train_acc = 0.7917
epoch : 15,	loss = 0.8238,	train_acc = 0.7917
epoch : 16,	loss = 0.7907,	train_acc = 0.7333
epoch : 17,	loss = 0.7691,	train_acc = 0.6833
epoch : 18,	loss = 0.7404,	train_acc = 0.6750
epoch : 19,	loss = 0.6843,	train_acc = 0.6750
epoch : 20,	loss = 0.7209,	train_acc = 0.6750
epoch : 21,	loss = 0.6936,	train_acc = 0.683

In [14]:
s = 0

for XXm, yym in dl_test :
    XXm = XXm.to("cuda:0")
    yym = yym.to("cuda:0")

    s += (net(XXm).argmax(axis = 1) == yym.argmax(axis = 1)).sum()

acc = s/len(yy)
print(f"test_acc = {acc:.4f}")

test_acc = 0.9667


`(5)` 아래의 제약사항에 맞추어 Species를 예측할 수 있는 적당한
네트워크를 학습하라.

**제약사항**

-   학습가능한 파라메터는 2층이상 설계할 것
-   드랍아웃을 포함시킬 것
-   학습 후 test accuracy 가 70% 이상일 것
-   train에서 `batch_size = 50` 으로, test에서는 `batch_size=30`을
    설정할것
-   GPU를 사용할 것
-   매 epoch마다 loss와 train accuracy를 출력할 것

In [16]:
## data
ds_train = torch.utils.data.TensorDataset(X, y)
dl_train = torch.utils.data.DataLoader(ds_train, batch_size = 50, shuffle = True)

ds_test = torch.utils.data.TensorDataset(XX, yy)
dl_test = torch.utils.data.DataLoader(ds_test, batch_size = 30, shuffle = True)

## network
net = torch.nn.Sequential(
    torch.nn.Linear(4, 1024),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(1024, 3)
).to("cuda:0")

loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters(), lr = 0.01)

##--##
for epoc in range(20) :
    net.train()

    for Xm, ym in dl_train :
        Xm = Xm.to("cuda:0")
        ym = ym.to("cuda:0")
        
        netout = net(Xm)
        loss = loss_fn(netout, ym)
        loss.backward()
        optimizr.step()
        optimizr.zero_grad()

    net.eval()

    s = 0
    
    for Xm, ym in dl_train :
        Xm = Xm.to("cuda:0")
        ym = ym.to("cuda:0")
        s += (net(Xm).argmax(axis = 1) == ym.argmax(axis = 1)).sum().item()

    acc = s/len(y)
    print(f"epoch : {epoc},\tloss = {loss:.4f},\ttrain_acc = {acc:.4f}")

epoch : 0,	loss = 5.2917,	train_acc = 0.6750
epoch : 1,	loss = 2.6907,	train_acc = 0.3750
epoch : 2,	loss = 1.2877,	train_acc = 0.6750
epoch : 3,	loss = 0.1650,	train_acc = 0.8917
epoch : 4,	loss = 0.6500,	train_acc = 0.7667
epoch : 5,	loss = 0.4570,	train_acc = 0.7167
epoch : 6,	loss = 0.0911,	train_acc = 0.9083
epoch : 7,	loss = 0.1824,	train_acc = 0.9250
epoch : 8,	loss = 0.1707,	train_acc = 0.8583
epoch : 9,	loss = 0.2091,	train_acc = 0.9750
epoch : 10,	loss = 0.1088,	train_acc = 0.9000
epoch : 11,	loss = 0.1407,	train_acc = 0.9417
epoch : 12,	loss = 0.3887,	train_acc = 0.9417
epoch : 13,	loss = 0.1715,	train_acc = 0.9083
epoch : 14,	loss = 0.1626,	train_acc = 0.9667
epoch : 15,	loss = 0.1006,	train_acc = 0.9417
epoch : 16,	loss = 0.1631,	train_acc = 0.9833
epoch : 17,	loss = 0.0529,	train_acc = 0.9750
epoch : 18,	loss = 0.0355,	train_acc = 0.9750
epoch : 19,	loss = 0.1224,	train_acc = 0.9833


In [17]:
s = 0

for XXm, yym in dl_test :
    XXm = XXm.to("cuda:0")
    yym = yym.to("cuda:0")

    s += (net(XXm).argmax(axis = 1) == yym.argmax(axis = 1)).sum()

acc = s/len(yy)
print(f"test_acc = {acc:.4f}")

test_acc = 0.9667
